In [1]:
import load_data

# Parse data from ../data/processed

In [2]:
reload(load_data)
pres_dict = load_data.read_processed_data("../data/processed/")
load_data.print_dict(pres_dict)

Processing ../data/processed/ ...
Donald J. Trump: vocab count 14307, sentence count 4385, word count 65436
Barack Obama: vocab count 15305, sentence count 3106, word count 64466
Abraham Lincoln: vocab count 15975, sentence count 2102, word count 66144


# Parse json data from ../data/unprocessed

In [3]:
load_data.read_unprocessed_data(pres_dict, "../data/unprocessed/")
load_data.print_dict(pres_dict)

Processing ../data/unprocessed/ ...
Lyndon B. Johnson: vocab count 118961, sentence count 22235, word count 445977
Gerald R. Ford: vocab count 34320, sentence count 6283, word count 135621
Richard Nixon: vocab count 40528, sentence count 7426, word count 191118
Franklin D. Roosevelt: vocab count 123191, sentence count 16084, word count 399627
William J. Clinton: vocab count 73996, sentence count 16625, word count 359552
Harry S. Truman: vocab count 139865, sentence count 31957, word count 413257
Abraham Lincoln: vocab count 15975, sentence count 2102, word count 66144
George Bush: vocab count 87076, sentence count 21374, word count 385669
John F. Kennedy: vocab count 61138, sentence count 11051, word count 258231
Dwight D. Eisenhower: vocab count 161616, sentence count 22835, word count 601774
Ronald Reagan: vocab count 48017, sentence count 8770, word count 192872
George W. Bush: vocab count 71606, sentence count 20821, word count 361515
Herbert Hoover: vocab count 74728, sentence cou

# Set up train and test data
This part of code will use num_of_words and the threshold of words to select from each president speeches. If a president doesn't have word counts (less than num_of_words), he will be skipped.

The data will be converted into 2d matrix give a batch size, and then split into 80% for training and 20% for test

In [4]:
batch_size = 100
num_of_words = 100000
president_int, y_train, X_train, y_test, X_test = load_data.create_train_test_data(pres_dict, num_of_words, batch_size)

# X_train.shape = (1407, 100): the number of original words were 140700. They were broken into batches of 100
# y_train.shape = (1407, 4): each batch is mapped to 1 of the 4 presidents
print X_train.shape, y_train.shape, X_test.shape, y_test.shape

Max number of words: 100000
Processing data for Lyndon B. Johnson
Processing data for Gerald R. Ford
Processing data for Richard Nixon
Processing data for Franklin D. Roosevelt
Processing data for William J. Clinton
Processing data for Harry S. Truman
Processing data for George Bush
Processing data for

load_data.py:115: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (a == None):


 John F. Kennedy
Processing data for Dwight D. Eisenhower
Processing data for Ronald Reagan
Processing data for George W. Bush
Processing data for Herbert Hoover
Processing data for Barack Obama
Processing data for Donald J. Trump
Processing data for Jimmy Carter
(12000, 100) (12000, 15) (3000, 100) (3000, 15)


# Create Keras Model as a baseline and Training

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils


# Instantiate and build model
model = Sequential()
model.add(Dense(units=400, input_dim=batch_size))
model.add(Activation('tanh'))
model.add(Dense(units=400, input_dim=200))
model.add(Activation('tanh'))
model.add(Dense(units=400, input_dim=200))
model.add(Activation('tanh'))
model.add(Dense(units=len(president_int.keys())))
model.add(Activation('softmax'))

# Compile w/ chosen loss, optimization fns; specific output metrics
model.compile(loss='categorical_crossentropy',
              optimizer='Adagrad',
              metrics=['categorical_accuracy','accuracy'])
# Train
print "Training..."
model.fit(X_train, y_train, epochs=60, batch_size=100)


Training...


Using TensorFlow backend.


Epoch 1/60
12000/12000 [==============================] - 0s - loss: 3.0406 - categorical_accuracy: 0.0710 - acc: 0.0710     

# Test and predict

In [6]:
# Evaluate performance
print "Evaluating test data..."
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)

# Make some predictions
print "\n\nPredicting using test data..."
predictions = model.predict(X_test, batch_size=128)

#print (classes)

print "\n\n",model.metrics_names
print loss_and_metrics


def prediction_to_01(predictions):
    max_p = predictions.max(axis=1)

    pred_int = None
    for i in range(len(max_p)):
        pred_int = load_data.append_matrices(pred_int, [(predictions[i] == max_p[i]).astype(int)])
    return pred_int

print prediction_to_01(predictions)
print y_test

Evaluating test data...
2048/3000 [===================>..........] - ETA: 0s

Predicting using test data...


['loss', 'categorical_accuracy', 'acc']
[4.8264018058776852, 0.073999999999999996, 0.073999999999999996]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
